# File Notification Mode

Service Principle Configuration

In [0]:
client_id = "XXXX-XXXX-XXXX-XXXX-XXXX-XXXX"
tenant_id = "XXXX-XXXX-XXXX-XXXX-XXXX-XXXX"
client_secret = "XXXX-XXXX-XXXX-XXXX-XXXX-XXXX"
storage_account_name = "dldatalakestorage"
container_name = "source"

In [0]:
spark.conf.set(f"fs.azure.account.auth.type.{storage_account_name}.dfs.core.windows.net", "OAuth")
spark.conf.set(f"fs.azure.account.oauth.provider.type.{storage_account_name}.dfs.core.windows.net", 
               "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set(f"fs.azure.account.oauth2.client.id.{storage_account_name}.dfs.core.windows.net", client_id)
spark.conf.set(f"fs.azure.account.oauth2.client.secret.{storage_account_name}.dfs.core.windows.net", client_secret)
spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{storage_account_name}.dfs.core.windows.net", 
               f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")

file_path = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net"

Also need to give secret values for event subscription verification

In [0]:
df = spark.readStream.format("cloudFiles") \
    .option("cloudFiles.format", "csv") \
    .option("cloudFiles.schemaLocation", f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/schema") \
    .option("cloudFiles.inferColumnTypes", "true") \
    .option("cloudFiles.useNotifications", "true") \
    .option("header", "true") \
    .option("cloudFiles.schemaEvolutionMode", "addNewColumns") \
    .option("cloudFiles.clientId", client_id) \
    .option("cloudFiles.clientSecret", client_secret) \
    .option("cloudFiles.tenantId", tenant_id) \
    .option("cloudFiles.subscriptionId", "dedd8773-e5a0-4cb7-88f0-86974468b0ca")\
    .option("cloudFiles.resourceGroup", "databricks_learning")\
    .load(file_path)


In [0]:
df.printSchema()


root
 |-- name: string (nullable = true)
 |-- city: string (nullable = true)
 |-- _rescued_data: string (nullable = true)



In [0]:
container_name = 'sink'

df.writeStream.format("delta") \
    .option("checkpointLocation", f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/checkpoint") \
    .outputMode("append") \
    .option("mergeSchema", "true")\
    .start(f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net")

In [0]:
df_sink = spark.read.format("delta").load(f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net")
df_sink.printSchema()

root
 |-- name: string (nullable = true)
 |-- city: string (nullable = true)
 |-- _rescued_data: string (nullable = true)



In [0]:
df_sink.show()

+---------+--------+-------------+
|     name|    city|_rescued_data|
+---------+--------+-------------+
|dhruvpuri|   surat|         NULL|
|    vivek|  rajkot|         NULL|
|      raj|junagadh|         NULL|
+---------+--------+-------------+

